# 心理咨询论坛问答语料 - 制作迁徙学习数据集

为 NVIDIA/Megatron-LM 准备该领域的 finetune 数据集

## 要点

- 使用教少量的心理咨询论坛问答数据，在之前的泛情绪类预训练模型基础上进行 finetune
- 使用特殊标记(Token) 分隔问题标题、文本、答案文本

## Importings

In [1]:
import os
import json
import random
from copy import copy
from contextlib import closing, ExitStack
from functools import partial
from glob import glob, iglob
from fileinput import FileInput
from itertools import chain, repeat
from multiprocessing import Pool

import pandas as pd
import sentencepiece as spm
from tqdm.auto import tqdm

from IPython.display import display

## Tokenizer

使用与 Pretrained-Model 同样的 tokenizer

In [2]:
spm_model_path = '../data/spm/gpt2_huamei_corpus_bpe_32k_v2.model'

tokenizer = spm.SentencePieceProcessor()
tokenizer.load(spm_model_path)

True

### 特殊标记

该 Tokenizer 既有的特殊标记有：

In [3]:
for i in range(8):
  print(i, tokenizer.id_to_piece(i), tokenizer.is_control(i))

0 <pad> True
1 <unk> False
2 <bos> True
3 <eos> True
4 <sep> False
5 <cls> False
6 <|endoftext|> False
7 一个 False


我们选用：

- 使用`id=4 <sep>` 表示问题部分中，标题与正文的分隔
- 使用`id=6 <|endoftext|>` 表示问题结束
- 使用 `id=2 <bos>` 表示回答的开始

In [4]:
SEP = tokenizer.id_to_piece(4)
EOT = tokenizer.id_to_piece(6)
BOS = tokenizer.id_to_piece(2)

## Corpus

In [5]:
corpus_files = glob('/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/*.json')

In [6]:
print('语料文件:')
display(corpus_files)
print()

it = FileInput(corpus_files)
total = sum(1 for _ in tqdm(it, unit='line'))

print(f'总行数: {total:,d}')

语料文件:


['/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/valid_xinli_qax.json',
 '/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/train_xinli_qax.json',
 '/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/test_xinli_qax.json']


总行数: 187,156


现在预览语料数据格式，随机选择一个：

In [7]:
n = random.randint(0, total)

it = enumerate(FileInput(corpus_files))
for i, line in tqdm(it, total=n):
    if i < n:
        continue
    data = json.loads(line)
    display(data)
    break


{'topics': ['恋爱', '出轨', '依赖依恋', '安全感'],
 'title': ['23岁女，看见有人向男友表白，不知道该不该怀疑他？'],
 'text': ['一个月前，在男友微信上看见有人跟他表白，看见聊天记录上那个人一直找他聊天，男友也会偶尔会回复一下，但聊的不是很多。',
  '但是后面就看见那个人跟他表白了，但是男友也没有拒绝的很明显，甚至没有表明自己有女朋友的这件事，询问后他解释说他们什么都没有，他也不喜欢他，也答应我说不会和她再聊天了。',
  '但是也改了手机密码，我不知道我是不是该怀疑他，他会出轨吗？'],
 'answers': [['对方会不会出轨这是难以预见的，',
   '仅仅这样的情况就判断会不会出轨那是言之过早。',
   '不管你选择相信还是怀疑，',
   '都需要对自己的选择负责。'],
  ['朋友你好，',
   '看到你的描述，',
   '大致了解到情况。',
   '不过在分析之前，',
   '我想先说明一点，',
   '我不能改变你的选择，',
   '我也不能够预知未来你的男友会不会出轨。',
   '我能够给你的是一个考虑这个问题，',
   '看待这个问题的角度。',
   '从你的描述中来看，',
   '你是一个理智且有包容心的人，',
   '看到有人和自己的男友聊天甚至有想要告白的企图，',
   '仍然可以好好说话，',
   '听男友的解释而不是大闹特闹，',
   '让男友把女孩的微信删掉。',
   '你并没有过多的干涉，',
   '这一点你处理地很好。',
   '给男友交友空间，',
   '并且愿意开诚布公聊这件事。',
   '另外你说到男友把手机密码改了，',
   '这一点我觉得是正常的，',
   '为啥这么说呢？',
   '因为现代社会，',
   '手机可以说是带锁的密码本，',
   '什么事情都可以在手机里找到答案，',
   '也很有可能过度解读一些事情。',
   '每个人都有保留自己的权利，',
   '人的两大基本需要亲密和自主。',
   '你的男友从你这里得到亲密，',
   '但他仍然有自主的权利。',
   '当然你也有，',
   '你可以选择怀疑他，',
   '强迫他把手机密码改回来，',
   '但你已经伤害到他的自主，'

## 按主题搜索

试试看，指定一个标签，看看有多少个样本

In [18]:
topic = '婚姻'

def filter_line_by_topic(line, topic):
    line = line.strip()
    if line:
        sample = json.loads(line)
        topics = sample.get('topics', [])
        if topic in topics:
            return True
    return False

it_lines = FileInput(corpus_files)
with Pool() as pool:
    it_map = pool.imap_unordered(
        partial(filter_line_by_topic, topic=topic),
        tqdm(it_lines, total=total),
        chunksize=min(1024, total//os.cpu_count()+1)
    )
    it_map = tqdm(it_map, total=total)
    c = sum(1 for x in it_map if x)

print(f'主题 {topic!r} 数量: {c:,d}, 占比: {c/total}')



主题 '婚姻' 数量: 31,760, 占比: 0.16969800594156748


## 样本选用和格式转换

我们应：

- 选取问题+回答构成输入样本
- 一个问题可以对应多个回答
- 不要太短的
- 不要太长的

在此基础上，选用 `10,000` 个样本用于今次的开发

In [21]:
### 长短限制定义：

MIN_QUESTION_LENGTH = 16
MAX_QUESTION_LENGTH = 512
MIN_ANSWER_LENGTH = 32
MAX_LENGTH = 1024+128


def extract_samples(line):
    line = line.strip()
    if not line:
        return []
    data = json.loads(line)
    result = []
    # 主题不对的不要！
    topics = data.get('topics', [])
    if topic not in topics:
        return []
    # 没有回答的不要
    if not data.get('answers'):
        return []
    # 问题:
    title = ''.join(data['title']).strip()
    # 没有标题的不要
    if not title:
        return []
    text = ''.join(data['text']).strip()
    # 没有问题内容的不要
    if not text:
        return []
    n_title = len(tokenizer.encode_as_ids(title))
    n_text = len(tokenizer.encode_as_ids(text))
    # 太短不要
    if n_title + n_text < MIN_QUESTION_LENGTH:
        return []
    # 太长不要
    if n_title + n_text > MAX_QUESTION_LENGTH:
        return []
    #
    question_string = title + SEP + text + EOT
    # 答案
    for answer in data.get('answers', []):
        text = ''.join(answer).strip()
        length = len(tokenizer.encode_as_ids(text))
        # 太短不要
        if length < MIN_ANSWER_LENGTH:
            continue
        # 太长不要
        if n_title + n_text + length > MAX_LENGTH:
            continue
        result.append(question_string + BOS + text)
    #
    return result

用上一次随机取得的行试试看：

In [22]:
extract_samples(line)

[]

## 计算全部样本

In [24]:
def generate_samples_from_corpus():
    lines_iter = FileInput(corpus_files)
    lines_iter = tqdm(lines_iter, desc='MAP', total=total)
    with Pool() as pool:
        mapping_iter = pool.imap_unordered(extract_samples, lines_iter, chunksize=min(1024, total//os.cpu_count())+1)
        mapping_iter = tqdm(mapping_iter, desc='RDC', total=total)
        yield from chain.from_iterable(mapping_iter)

samples = []
for sample in generate_samples_from_corpus():
    samples.append(sample)

print(f'满足要求的样本数: {len(samples):,d}')



满足要求的样本数: 24,768


随机显示一个:

In [31]:
s = random.choice(samples)
print(s)

老婆单位受气，容易找我撒气，我该如何处理为好？<sep>老婆在单位和直接领导做事想法、方式差异较大，看不顺眼，负面情绪容易带到家里，拿我撒气，开始我不清楚，拿来撒气的小事会辩解，惹来更大的脾气就会觉得很无语，后来细想后有些了解了，也就不做啥辩解了，但是是否有好点的处理方法呢？家里可否少些这种情绪宣泄？<|endoftext|><bos>人在熟悉安全的环境才会释放自我，很明显你老婆找你撒气，是希望得到你的关注，认同，理解她。和她站在一条战线上。发脾气，生气，这是我们每个人都会有的体验和经历，它是一种硬情绪，是自我保护的一种方式。在生气、愤怒的背后往往隐藏着软情绪，比如悲伤、失落，这些是我们内在的脆弱，是不太愿意被别人看到的部分，所以往往会通过发脾气的方式来表达不满，而更深层次的原因是我们感觉到没有被看见、被尊重、被支持、被关爱、被认可。在这世界上，没有人想给别人惹麻烦，想惹别人生气。如果你认识到每个人都是从自己的角度来解读世界，每一个愤怒、发脾气、生气的背后，都是对爱的呼唤，就能更容易理解别人了。情绪如何处理为好？如果你能够理解，你就多帮帮她说话，站在她那边，替她想一想。看看能不能给出一些建议。其实情绪会告诉我们一些什么，每个情绪都是我们的朋友。1、给情绪命名。能够给情绪命名，就能化解情绪。感受一下自己或她的情绪，确定自己或她的情绪是生气、愤怒、还是烦躁，然后告诉自己我此刻很怎么样。比如生气2、（情绪的核心）我真正的需求是什么。继续保持深呼吸，和自己对话：我到底为什么生气，我真正需要的是什么，是因为我不被尊重？不被重视？不被理解？不被关爱？还是自己的悲伤、失落、恐惧等情绪干扰了我的心智？到底什么是真相，到底什么是我真正的需求？3、有时间就接你老婆下班，如果她有情绪可以陪她在外面喊山或其它方式来减压，发泄完了再回家，这样可以减少在家宣泄情绪。当然如果你有时间可以把家里的卫生打扫一下，把东西摆放整齐，当她看到家里干净的环境，也会让她有一个好的心情。


## 采样并存储

划分为 train/val/test

In [34]:
part_num = {
    'train': 20000,
    'val': 1000,
    'test': 1000,
}

print('数据集划分：')
display(part_num)

数据集划分：


{'train': 20000, 'val': 1000, 'test': 1000}

随机种子:

In [35]:
%%time

seeds = []
seeds.extend(chain.from_iterable(
    repeat(k, v) for k, v in part_num.items()
))
seeds.extend(repeat(None, len(samples)-sum(part_num.values())))
random.shuffle(seeds)

assert len(seeds) == len(samples)

CPU times: user 19.1 ms, sys: 648 µs, total: 19.8 ms
Wall time: 18.9 ms


保存为 NVIDIA/Megatron-LM 的 Loose Json 格式，使用 `"text"` 作为 Key。

计算文件路径名：

In [41]:
%%time

from slugify import slugify

topic_slug = slugify(topic, separator='')

output_files = {
    k: os.path.join('..', 'data', f'xinliqa_{topic_slug}.{k}.json')
    for k in part_num.keys()
}

print('输出文件:')
display(output_files)

输出文件:


{'train': '../data/xinliqa_hunyin.train.json',
 'val': '../data/xinliqa_hunyin.val.json',
 'test': '../data/xinliqa_hunyin.test.json'}

CPU times: user 2.78 ms, sys: 822 µs, total: 3.6 ms
Wall time: 2.21 ms


输出到目标文件

In [42]:
lines_iter = FileInput(corpus_files)

with ExitStack() as stack:
    files = {
        k: stack.enter_context(open(v, 'w'))
        for k, v in output_files.items()
    }
    for seed, string in tqdm(zip(seeds, samples), total=len(samples)):
        fp = files.get(seed)
        if not fp:
            continue
        print(
            json.dumps({'text': string}, ensure_ascii=False),
            file=fp
        )

检查输出文件

In [43]:
s = ' '.join(output_files.values())

!wc -l {s}
print()

!du -h {s}
print()

   20000 ../data/xinliqa_hunyin.train.json
    1000 ../data/xinliqa_hunyin.val.json
    1000 ../data/xinliqa_hunyin.test.json
   22000 总用量

25M	../data/xinliqa_hunyin.train.json
1.3M	../data/xinliqa_hunyin.val.json
1.3M	../data/xinliqa_hunyin.test.json

